In [1]:
import geohash
from datetime import datetime as dt
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns
import os
import time
# import tensorflow as tf
# import utm
import gc
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.cluster.hierarchy import dendrogram, linkage
# from statsmodels.tsa.tsatools import lagmat
from functools import reduce
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# sns.set_style("darkgrid")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [2]:
# import lightgbm as lgb
# import xgboost as xgb

In [3]:
this_dir = os.path.abspath('')
feat_dir = os.path.join(this_dir, "new_feats_13_June")
# data_train_path = os.path.join(this_dir, "train_new_mod_31.csv")
# data_test_path = os.path.join(this_dir, "test_new_mod_31.csv")
data_train_path = os.path.join(feat_dir, "train.csv")
data_val_path = os.path.join(feat_dir, "val.csv")
data_test_path = os.path.join(feat_dir, "test.csv")

# data_test = pd.read_csv(data_test_path)

In [4]:
data_train = pd.read_csv(data_train_path)
data_val = pd.read_csv(data_val_path)
data_test = pd.read_csv(data_test_path)

In [5]:
data_train = data_train.sort_values(by = ['geohash6', 'day','time'])
data_train = data_train.reset_index(drop=True)

In [ ]:
# data_train.head()

In [6]:
tmp = data_train[['day','time', 'geohash6', 'demand']].copy()
tmp = tmp.groupby('geohash6')['demand']

In [7]:
series_train_lag_diff =  tmp.transform(lambda x: x.shift(1))

In [8]:
series_train_lag_diff = series_train_lag_diff.fillna(0)

In [9]:
series_train_rolling_diff = tmp.transform(lambda x: x.rolling(3).mean())
series_train_rolling_diff = series_train_rolling_diff.fillna(0)

In [10]:
data_train['u_diff_lag'] =  series_train_lag_diff - data_train['mean_demand_per_hour']
data_train['median_diff_lag'] =  series_train_lag_diff - data_train['median_demand_per_hour']
# ---------------------
data_train['u_diff_lag_week'] =  series_train_lag_diff - data_train['mean_demand_per_week']
data_train['median_diff_lag_week'] =  series_train_lag_diff - data_train['median_demand_per_week']
# ---------------------
data_train['u_diff_lag_geo'] =  series_train_lag_diff - data_train['mean_demand_per_geo']
data_train['median_diff_lag_geo'] =  series_train_lag_diff - data_train['median_demand_per_geo']
# --------------------
data_train['u_diff_rolling'] = series_train_rolling_diff - data_train['mean_demand_per_hour']
data_train['u_diff_rolling_week'] = series_train_rolling_diff - data_train['mean_demand_per_week']
data_train['u_diff_rolling_geo'] = series_train_rolling_diff - data_train['mean_demand_per_geo']

In [12]:
data_train.head()

,day,time,geohash6,demand,consec_zeros,week_day,hour,minute,mean_demand_per_hour,median_demand_per_hour,mean_demand_per_week,median_demand_per_week,mean_demand_per_geo,median_demand_per_geo,latitude,longitude,u_diff_lag,median_diff_lag,u_diff_lag_week,median_diff_lag_week,u_diff_lag_geo,median_diff_lag_geo,u_diff_rolling,u_diff_rolling_week,u_diff_rolling_geo
0,1,00:00:00,qp02yc,0.0,0.0,1,0,0,0.0,0.0,0.000923,0.0,0.000428,0.0,-5.484924,90.653687,0.0,0.0,-0.000923,0.0,-0.000428,0.0,0.0,-0.000923,-0.000428
1,1,00:15:00,qp02yc,0.0,1.0,1,0,15,0.0,0.0,0.000923,0.0,0.000428,0.0,-5.484924,90.653687,0.0,0.0,-0.000923,0.0,-0.000428,0.0,0.0,-0.000923,-0.000428
2,1,00:30:00,qp02yc,0.0,2.0,1,0,30,0.0,0.0,0.000923,0.0,0.000428,0.0,-5.484924,90.653687,0.0,0.0,-0.000923,0.0,-0.000428,0.0,0.0,-0.000923,-0.000428
3,1,00:45:00,qp02yc,0.0,3.0,1,0,45,0.0,0.0,0.000923,0.0,0.000428,0.0,-5.484924,90.653687,0.0,0.0,-0.000923,0.0,-0.000428,0.0,0.0,-0.000923,-0.000428
4,1,01:00:00,qp02yc,0.0,4.0,1,1,0,0.0,0.0,0.000923,0.0,0.000973,0.0,-5.484924,90.653687,0.0,0.0,-0.000923,0.0,-0.000973,0.0,0.0,-0.000923,-0.000973


## Diff Test

In [11]:
data_test.head()

,geohash6,day,demand,time,week_day,hour,minute,consec_zeros,mean_demand_per_hour,median_demand_per_hour,median_demand_per_week,mean_demand_per_week,median_demand_per_geo,mean_demand_per_geo,latitude,longitude
0,qp03nr,20,0.000293,20:45:00,2,20,45,1.0,0.003644,0.001754,0.083711,0.093018,0.000233,0.003950,-5.408020,90.631714
1,qp093r,17,0.054170,06:15:00,6,6,15,0.0,0.082672,0.081182,0.046492,0.045976,0.082631,0.085514,-5.364075,90.763550
2,qp03rp,21,0.359406,11:45:00,3,11,45,0.0,0.141845,0.146364,0.077169,0.086033,0.115335,0.119695,-5.364075,90.664673
3,qp03w9,15,0.514136,12:15:00,4,12,15,0.0,1.000000,1.000000,0.939273,0.788614,1.000000,0.782725,-5.353088,90.642700
4,qp03mf,17,0.057255,11:30:00,6,11,30,0.0,0.029496,0.031075,0.021468,0.023118,0.031409,0.032170,-5.391541,90.609741


In [14]:
data_test = data_test.sort_values(by = ['geohash6', 'day','time'])
data_test = data_test.reset_index(drop=True)

In [15]:
lag_feat = data_test[['day', 'time', 'geohash6', 'demand']].copy()
lag_feat = lag_feat.set_index(['day', 'time', 'geohash6'])
lag_feat = lag_feat.unstack().shift(1) # pull out the groups, shift with lag step=1
lag_feat = lag_feat.stack(dropna=False)
lag_feat = lag_feat.reset_index()
lag_feat.rename(columns={'demand': 'lag_demand_1'}, inplace = True)

In [16]:
tmp2 = pd.merge(data_test, lag_feat, how = 'inner', on = ['day', 'time' , 'geohash6'])

In [17]:
tmp2['lag_demand_1'] = tmp2['lag_demand_1'].fillna(0)

In [18]:
series_test_lag_diff = tmp2['lag_demand_1']

In [ ]:
# tmp2 = data_test[['day','time', 'geohash6', 'demand']].copy()
# tmp2 = tmp2.groupby('geohash6')['time','demand']

In [ ]:
# series_test_lag_diff = tmp2.transform(lambda x: x.shift(1))
# series_test_lag_diff = series_test_lag_diff.fillna(0)

In [ ]:
# series_test_rolling_diff = tmp2.transform(lambda x: x.rolling(3).mean())
# series_test_rolling_diff = series_test_rolling_diff.fillna(0)

In [19]:
del tmp,tmp2
gc.collect()

32

In [20]:
roll_feat = data_test[['day', 'time', 'geohash6', 'demand']].copy()
roll_feat = roll_feat.set_index(['day', 'time', 'geohash6'])
roll_feat = roll_feat.unstack().rolling(3).mean() # pull out the groups, shift with lag step=1
roll_feat = roll_feat.stack(dropna=False)
roll_feat = roll_feat.reset_index()
roll_feat.rename(columns={'demand': 'roll_demand'}, inplace = True)

In [21]:
tmp3 = pd.merge(data_test, roll_feat, how = 'inner', on = ['day', 'time' , 'geohash6'])

In [22]:
tmp3['roll_demand'] = tmp3['roll_demand'].fillna(0)

In [23]:
series_test_rolling_diff = tmp3['roll_demand']

In [24]:
data_test['u_diff_lag'] =  series_test_lag_diff - data_test['mean_demand_per_hour']
data_test['median_diff_lag'] =  series_test_lag_diff - data_test['median_demand_per_hour']
# ----------------------
data_test['u_diff_lag_week'] =  series_test_lag_diff - data_test['mean_demand_per_week']
data_test['median_diff_lag_week'] =  series_test_lag_diff - data_test['median_demand_per_week']
# ---------------------
data_test['u_diff_lag_geo'] =  series_test_lag_diff - data_test['mean_demand_per_geo']
data_test['median_diff_lag_geo'] =  series_test_lag_diff - data_test['median_demand_per_geo']
# ----------------------
data_test['u_diff_rolling'] = series_test_rolling_diff - data_test['mean_demand_per_hour']
data_test['u_diff_rolling_week'] = series_test_rolling_diff - data_test['mean_demand_per_week']
data_test['u_diff_rolling_geo'] = series_test_rolling_diff - data_test['mean_demand_per_geo']

In [ ]:
# # --------------- DROP TRASH ---------------
# data_train = data_train.drop(['mean_demand_per_week','median_demand_per_week', 'mean_demand_per_geo', 'median_demand_per_geo'],axis=1)
# data_test = data_test.drop(['mean_demand_per_week','median_demand_per_week', 'mean_demand_per_geo', 'median_demand_per_geo'],axis=1)

In [25]:
del tmp3, roll_feat, lag_feat
gc.collect()

84

In [ ]:
data_train.head()

In [26]:
data_test.head()

,geohash6,day,demand,time,week_day,hour,minute,consec_zeros,mean_demand_per_hour,median_demand_per_hour,median_demand_per_week,mean_demand_per_week,median_demand_per_geo,mean_demand_per_geo,latitude,longitude,u_diff_lag,median_diff_lag,u_diff_lag_week,median_diff_lag_week,u_diff_lag_geo,median_diff_lag_geo,u_diff_rolling,u_diff_rolling_week,u_diff_rolling_geo
0,qp02yc,15,0.010794,01:00:00,4,1,0,4.0,0.000234,0.0,0.0,0.001775,0.0,0.000973,-5.484924,90.653687,-0.000234,0.000000,-0.001775,0.000000,-0.000973,0.000000,-0.000234,-0.001775,-0.000973
1,qp02yc,15,0.039689,03:45:00,4,3,45,15.0,0.003458,0.0,0.0,0.001775,0.0,0.000963,-5.484924,90.653687,-0.003458,0.000000,-0.001775,0.000000,-0.000963,0.000000,-0.003458,-0.001775,-0.000963
2,qp02yc,15,0.016503,04:30:00,4,4,30,18.0,0.007949,0.0,0.0,0.001775,0.0,0.005102,-5.484924,90.653687,-0.007949,0.000000,-0.001775,0.000000,-0.005102,0.000000,-0.007949,-0.001775,-0.005102
3,qp02yc,15,0.022289,05:15:00,4,5,15,1.0,0.003452,0.0,0.0,0.001775,0.0,0.005254,-5.484924,90.653687,-0.003452,0.000000,-0.001775,0.000000,-0.005254,0.000000,-0.003452,-0.001775,-0.005254
4,qp02yc,15,0.019722,05:30:00,4,5,30,0.0,0.003452,0.0,0.0,0.001775,0.0,0.005254,-5.484924,90.653687,0.018837,0.022289,0.020515,0.022289,0.017035,0.022289,-0.003452,-0.001775,-0.005254


## Diff Val

In [28]:
data_val.head()

,geohash6,day,demand,time,week_day,hour,minute,consec_zeros,mean_demand_per_hour,median_demand_per_hour,median_demand_per_week,mean_demand_per_week,median_demand_per_geo,mean_demand_per_geo,latitude,longitude
0,qp0991,1,0.088755,05:00:00,4,5,0,0.0,0.091747,0.082787,0.061154,0.061942,0.098566,0.100854,-5.353088,90.752563
1,qp09kb,14,0.026177,09:15:00,3,9,15,0.0,0.033773,0.032421,0.000000,0.011149,0.025281,0.027753,-5.402527,90.917358
2,qp096m,3,0.059740,14:45:00,6,14,45,0.0,0.040315,0.034057,0.034993,0.036198,0.061952,0.065867,-5.375061,90.807495
3,qp091u,9,0.026409,02:30:00,5,2,30,0.0,0.003763,0.001177,0.000000,0.002861,0.000000,0.006662,-5.424500,90.785522
4,qp09bd,5,0.040678,05:00:00,1,5,0,0.0,0.069703,0.063536,0.055372,0.078994,0.076552,0.078258,-5.303650,90.730591


In [29]:
data_val = data_val.sort_values(by = ['geohash6', 'day','time'])
data_val = data_val.reset_index(drop=True)

In [30]:
lag_feat = data_val[['day', 'time', 'geohash6', 'demand']].copy()
lag_feat = lag_feat.set_index(['day', 'time', 'geohash6'])
lag_feat = lag_feat.unstack().shift(1) # pull out the groups, shift with lag step=1
lag_feat = lag_feat.stack(dropna=False)
lag_feat = lag_feat.reset_index()
lag_feat.rename(columns={'demand': 'lag_demand_1'}, inplace = True)

In [31]:
tmp2 = pd.merge(data_val, lag_feat, how = 'inner', on = ['day', 'time' , 'geohash6'])

In [32]:
tmp2['lag_demand_1'] = tmp2['lag_demand_1'].fillna(0)

In [33]:
series_test_lag_diff = tmp2['lag_demand_1']

In [34]:
# tmp2 = data_test[['day','time', 'geohash6', 'demand']].copy()
# tmp2 = tmp2.groupby('geohash6')['time','demand']

In [35]:
# series_test_lag_diff = tmp2.transform(lambda x: x.shift(1))
# series_test_lag_diff = series_test_lag_diff.fillna(0)

In [36]:
# series_test_rolling_diff = tmp2.transform(lambda x: x.rolling(3).mean())
# series_test_rolling_diff = series_test_rolling_diff.fillna(0)

In [38]:
del tmp2
gc.collect()

7

In [39]:
roll_feat = data_val[['day', 'time', 'geohash6', 'demand']].copy()
roll_feat = roll_feat.set_index(['day', 'time', 'geohash6'])
roll_feat = roll_feat.unstack().rolling(3).mean() # pull out the groups, shift with lag step=1
roll_feat = roll_feat.stack(dropna=False)
roll_feat = roll_feat.reset_index()
roll_feat.rename(columns={'demand': 'roll_demand'}, inplace = True)

In [40]:
tmp3 = pd.merge(data_val, roll_feat, how = 'inner', on = ['day', 'time' , 'geohash6'])

In [41]:
tmp3['roll_demand'] = tmp3['roll_demand'].fillna(0)

In [42]:
series_test_rolling_diff = tmp3['roll_demand']

In [43]:
data_val['u_diff_lag'] =  series_test_lag_diff - data_val['mean_demand_per_hour']
data_val['median_diff_lag'] =  series_test_lag_diff - data_val['median_demand_per_hour']
# ----------------------
data_val['u_diff_lag_week'] =  series_test_lag_diff - data_val['mean_demand_per_week']
data_val['median_diff_lag_week'] =  series_test_lag_diff - data_val['median_demand_per_week']
# ---------------------
data_val['u_diff_lag_geo'] =  series_test_lag_diff - data_val['mean_demand_per_geo']
data_val['median_diff_lag_geo'] =  series_test_lag_diff - data_val['median_demand_per_geo']
# ----------------------
data_val['u_diff_rolling'] = series_test_rolling_diff - data_val['mean_demand_per_hour']
data_val['u_diff_rolling_week'] = series_test_rolling_diff - data_val['mean_demand_per_week']
data_val['u_diff_rolling_geo'] = series_test_rolling_diff - data_val['mean_demand_per_geo']

In [ ]:
# # --------------- DROP TRASH ---------------
# data_train = data_train.drop(['mean_demand_per_week','median_demand_per_week', 'mean_demand_per_geo', 'median_demand_per_geo'],axis=1)
# data_test = data_test.drop(['mean_demand_per_week','median_demand_per_week', 'mean_demand_per_geo', 'median_demand_per_geo'],axis=1)

In [44]:
del tmp3, roll_feat, lag_feat
gc.collect()

84

In [45]:
data_val.head()

,geohash6,day,demand,time,week_day,hour,minute,consec_zeros,mean_demand_per_hour,median_demand_per_hour,median_demand_per_week,mean_demand_per_week,median_demand_per_geo,mean_demand_per_geo,latitude,longitude,u_diff_lag,median_diff_lag,u_diff_lag_week,median_diff_lag_week,u_diff_lag_geo,median_diff_lag_geo,u_diff_rolling,u_diff_rolling_week,u_diff_rolling_geo
0,qp02yc,1,0.010710,04:15:00,4,4,15,0.0,0.007949,0.0,0.0,0.001775,0.0,0.005102,-5.484924,90.653687,-0.007949,0.000000,-0.001775,0.000000,-0.005102,0.000000,-0.007949,-0.001775,-0.005102
1,qp02yc,1,0.000642,04:30:00,4,4,30,1.0,0.007949,0.0,0.0,0.001775,0.0,0.005102,-5.484924,90.653687,0.002762,0.010710,0.008935,0.010710,0.005608,0.010710,-0.007949,-0.001775,-0.005102
2,qp02yc,1,0.021575,04:45:00,4,4,45,0.0,0.007949,0.0,0.0,0.001775,0.0,0.005102,-5.484924,90.653687,-0.007306,0.000642,-0.001133,0.000642,-0.004460,0.000642,0.003027,0.009201,0.005874
3,qp02yc,1,0.000872,05:00:00,4,5,0,1.0,0.003452,0.0,0.0,0.001775,0.0,0.005254,-5.484924,90.653687,0.018123,0.021575,0.019801,0.021575,0.016321,0.021575,0.004244,0.005922,0.002443
4,qp02yc,1,0.037952,05:15:00,4,5,15,2.0,0.003452,0.0,0.0,0.001775,0.0,0.005254,-5.484924,90.653687,-0.002580,0.000872,-0.000902,0.000872,-0.004382,0.000872,0.016681,0.018358,0.014879


In [ ]:
# cols =  data_train.columns.values
# data_test = data_test[cols]

In [ ]:
# X_train = data_train.drop(['geohash6','time','demand'],axis=1)
# y_train = data_train['demand']
# X_test = data_test.drop(['geohash6','time','demand'],axis=1)
# y_test = data_test['demand']

In [ ]:
# X_train = X_train.astype(float)
# X_test = X_test.astype(float)

In [ ]:
# lgb_train_set = lgb.Dataset(X_train,y_train)
# lgb_test_set = lgb.Dataset(X_test,y_test)

In [ ]:
# params = {
#     'nthread': 10, 
#     'max_depth': 5,
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression_l2',
#     'metric': 'l2', 
#     'num_leaves': 64,
#     'learning_rate': 0.2,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'lambda_l1': 3.097758978478437,
#     'lambda_l2': 2.9482537987198496,
#     'verbose': 1,
# }

In [ ]:
# eval_results = {}

In [ ]:
# %%time
# reg = lgb.train(params, lgb_train_set, 500, 
#                 valid_sets=[lgb_train_set, lgb_test_set],
#                 evals_result= eval_results,
# #                 categorical_feature=['geohash6_numeric','neighbor_1', 'neighbor_2', 'neighbor_3', 'neighbor_4'],
#                 early_stopping_rounds=50)

#### [500]	training's l2: 0.000214491	valid_1's l2: 0.00065697
#### [324]	training's l2: 0.00022185	valid_1's l2: 0.000381518 new_mod_31 fill na on train and test to be 0

In [ ]:
# ax = lgb.plot_importance(reg, max_num_features=35, importance_type='gain')
# plt.show()

In [ ]:
# ax = lgb.plot_metric(eval_results, metric='l2')
# plt.show()

In [ ]:
# del reg
# gc.collect()

In [ ]:
# xgb_train_set = xgb.DMatrix(X_train,y_train)
# xgb_test_set = xgb.DMatrix(X_test,y_test)

In [ ]:
# data_train.to_csv('./train_new_feats_2.csv', index= False)
# data_test.to_csv('./test_new_feats_2.csv', index = False)

In [46]:
# data_train.to_csv(os.path.join(feat_dir, "train_diff_feats.csv"), index=False)
# data_val.to_csv(os.path.join(feat_dir, "val_diff_feats.csv"), index=False)
# data_test.to_csv(os.path.join(feat_dir, "test_diff_feats.csv"), index=False)

In [ ]:
# %%time
# xgb_reg = xgb.XGBRegressor(n_estimators=100, eta=0.2, objective = 'reg:squarederror', booster ='dart')
# xgb_reg.fit(X_train, y_train, 
#             eval_set=[(X_train, y_train), (X_test, y_test)], 
#             eval_metric = 'rmse', early_stopping_rounds=10)

In [ ]:
# xgb.plot_importance(xgb_reg,importance_type='gain', max_num_features=20)
# plt.show()